### Spark SQL

Pacotes adicionais do Spark: https://spark-packages.org

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql import Row

print('Verificando o contexto em que se encontra a conexão:', '\n')
print(sc) #sc = spark context
print('\n', 'Versão do SPARK em execução:', sc.version)

Verificando o contexto em que se encontra a conexão: 

<SparkContext master=local[*] appName=PySparkShell>

 Versão do SPARK em execução: 2.2.0


### SparkContext -> conexão Python-Spark (pelo Jupyter é criado automáticamente com a chamado do PySpark)

Em aplicações profissionais (deploy de solução em PRODUÇÃO), é necessário CONSTRUIR o CONTEXTO (explicitamente).

### SparkSession ->>> trabalhar com DataFrames no SparkSQL, necessário CONSTRUIR uma SESSÃO (explicitamente)...

In [3]:
spSession = SparkSession.builder.master("local").appName("POC-SparkSQL")\
          .config("spark.some.config.option", "some-value").getOrCreate()

### SQLContext ->>> trabalhar com LINGUAGEM SQL no SparkSQL, necessário CONSTRUIR o CONTEXTO (explicitamente)...

In [4]:
sqlSession = SQLContext(sc)

### Importando arquivo CSV e criando a RDD...

In [6]:
#file ="/home/daniellj/Projetos/Github/DistributedComputing/ApacheSpark/datasets/cars.csv"
file = "C:\\Users\\whitecube.daniel\\Projetos_Daniel\\DistributedComputing\\Datasets\\cars.csv"
carRDD = sc.textFile(file)

print('Tipo do objeto criado <carRDD>:', type(carRDD))
print('Quantidade de elementos do objeto RDD criado <carRDD>', carRDD.count(), '\n')
print('Exibindo as 5 primeiras linhas do RDD criado <carRDD>...', '\n',carRDD.take(5),'\n')

Tipo do objeto criado <carRDD>: <class 'pyspark.rdd.RDD'>
Quantidade de elementos do objeto RDD criado <carRDD> 198 

Exibindo as 5 primeiras linhas do RDD criado <carRDD>... 
 ['MAKE,FUELTYPE,ASPIRE,DOORS,BODY,DRIVE,CYLINDERS,HP,RPM,MPG-CITY,MPG-HWY,PRICE', 'subaru,gas,std,two,hatchback,fwd,four,69,4900,31,36,5118', 'chevrolet,gas,std,two,hatchback,fwd,three,48,5100,47,53,5151', 'mazda,gas,std,two,hatchback,fwd,four,68,5000,30,31,5195', 'toyota,gas,std,two,hatchback,fwd,four,62,4800,35,39,5348'] 



### RDD é um repositório de dados genéricos. É possível colocar qualquer coisa dentro de uma RDD.

In [7]:
rowRDD2 = carRDD.filter(lambda x : 'PRICE' not in x)
print('Quantidade de elementos onde não há linhas com a expressão "PRICE":', rowRDD2.count(), '\n')

print('Ajustando os dados com quebras a partir das vírgulas...exibindo as primeiras linhas após o ajustes...')
rowRDD3 = rowRDD2.map(lambda x: x.split(','))
rowRDD3.take(2)

Quantidade de elementos onde não há linhas com a expressão "PRICE": 197 

Ajustando os dados com quebras a partir das vírgulas...exibindo as primeiras linhas após o ajustes...


[['subaru',
  'gas',
  'std',
  'two',
  'hatchback',
  'fwd',
  'four',
  '69',
  '4900',
  '31',
  '36',
  '5118'],
 ['chevrolet',
  'gas',
  'std',
  'two',
  'hatchback',
  'fwd',
  'three',
  '48',
  '5100',
  '47',
  '53',
  '5151']]

In [36]:
?Row

In [ ]:
Row can be used to create a row object by using named arguments,
the fields will be sorted by names. It is not allowed to omit
a named argument to represent the value is None or missing. This should be
explicitly set to None in this case.

>>> row = Row(name="Alice", age=11)
>>> row
Row(age=11, name='Alice')

### Função Row implementada à seguir: divide o RDD e transforma cada linha em um objeto INDEPENDENTE.

In [8]:
rowRDD4 = rowRDD3.map(lambda x: Row(make = x[0], body = x[4], hp = x[7]))

In [9]:
print('Exibindo as 20 primeiras linhas independentes criadas <rowRDD4>...')
print('Observe que a ordenação dos parâmetros se dá por ordem alfabética.')
print('A ordenação dos registros segue o estado original...', '\n')
rowRDD4.take(20)

Exibindo as 20 primeiras linhas independentes criadas <rowRDD4>...
Observe que a ordenação dos parâmetros se dá por ordem alfabética.
A ordenação dos registros segue o estado original... 



[Row(body='hatchback', hp='69', make='subaru'),
 Row(body='hatchback', hp='48', make='chevrolet'),
 Row(body='hatchback', hp='68', make='mazda'),
 Row(body='hatchback', hp='62', make='toyota'),
 Row(body='hatchback', hp='68', make='mitsubishi'),
 Row(body='hatchback', hp='60', make='honda'),
 Row(body='sedan', hp='69', make='nissan'),
 Row(body='hatchback', hp='68', make='dodge'),
 Row(body='hatchback', hp='68', make='plymouth'),
 Row(body='hatchback', hp='68', make='mazda'),
 Row(body='hatchback', hp='68', make='mitsubishi'),
 Row(body='hatchback', hp='68', make='dodge'),
 Row(body='hatchback', hp='68', make='plymouth'),
 Row(body='hatchback', hp='70', make='chevrolet'),
 Row(body='hatchback', hp='62', make='toyota'),
 Row(body='hatchback', hp='68', make='dodge'),
 Row(body='hatchback', hp='58', make='honda'),
 Row(body='hatchback', hp='62', make='toyota'),
 Row(body='hatchback', hp='76', make='honda'),
 Row(body='sedan', hp='70', make='chevrolet')]

 ### Criando um dataframe, a partir de uma RDD, usando a SPARK CONTEXT criada anteriormente...
 
 Internamente, RDD = DataFrame.
 Vantagem dos DataFrames: maior facilidade de manipular dados.

In [14]:
rowRDD5 = spSession.createDataFrame(rowRDD4)
print('Tipo do objeto criado <rowRDD5>:', type(rowRDD5), '\n')

print('Imprimindo as 15 primeiras linhas do DataFrame criado...', '\n')
rowRDD5.show(15)

Tipo do objeto criado <rowRDD5>: <class 'pyspark.sql.dataframe.DataFrame'> 

Imprimindo as 15 primeiras linhas do DataFrame criado... 

+---------+---+----------+
|     body| hp|      make|
+---------+---+----------+
|hatchback| 69|    subaru|
|hatchback| 48| chevrolet|
|hatchback| 68|     mazda|
|hatchback| 62|    toyota|
|hatchback| 68|mitsubishi|
|hatchback| 60|     honda|
|    sedan| 69|    nissan|
|hatchback| 68|     dodge|
|hatchback| 68|  plymouth|
|hatchback| 68|     mazda|
|hatchback| 68|mitsubishi|
|hatchback| 68|     dodge|
|hatchback| 68|  plymouth|
|hatchback| 70| chevrolet|
|hatchback| 62|    toyota|
+---------+---+----------+
only showing top 15 rows



### SparkSQL command - Fazendo manipulações de dados do DataFrame criado em SparkSQL...

In [16]:
print('Consulta com select...', '\n')
rowRDD5.select('*').show(10)

Consulta com select... 

+---------+---+----------+
|     body| hp|      make|
+---------+---+----------+
|hatchback| 69|    subaru|
|hatchback| 48| chevrolet|
|hatchback| 68|     mazda|
|hatchback| 62|    toyota|
|hatchback| 68|mitsubishi|
|hatchback| 60|     honda|
|    sedan| 69|    nissan|
|hatchback| 68|     dodge|
|hatchback| 68|  plymouth|
|hatchback| 68|     mazda|
+---------+---+----------+
only showing top 10 rows



### SparkSQL command - Ordenando os dados do DataFrame por uma das colunas...

In [17]:
rowRDD5.orderBy('make').show()

+-----------+---+-----------+
|       body| hp|       make|
+-----------+---+-----------+
|  hatchback|154|alfa-romero|
|convertible|111|alfa-romero|
|convertible|111|alfa-romero|
|      sedan|110|       audi|
|      sedan|115|       audi|
|      sedan|110|       audi|
|      wagon|110|       audi|
|      sedan|140|       audi|
|      sedan|102|       audi|
|      sedan|101|        bmw|
|      sedan|101|        bmw|
|      sedan|121|        bmw|
|      sedan|121|        bmw|
|      sedan|182|        bmw|
|      sedan|182|        bmw|
|      sedan|121|        bmw|
|      sedan|182|        bmw|
|      sedan| 70|  chevrolet|
|  hatchback| 70|  chevrolet|
|  hatchback| 48|  chevrolet|
+-----------+---+-----------+
only showing top 20 rows



### Quando queremos utilizar a LINGUAGEM SQL, convertemos o conteúdo de origem (no cenário atual o DataFrame) para uma TABELA TEMPORÁRIA
### Convertendo...

In [30]:
rowRDD5.createOrReplaceTempView('rowTB1')

### A partir do momento que temos a tabela temporária criada, podemos disparar comandos SQL ANSI contra este objeto.

In [31]:
print('Disparando uma consulta contra a tabela temporária <rowTB1>...', '\n')

print('Exibindo todos os registros que sejam da marca "honda"...', '\n')
spSession.sql('select * from rowTB1 where make = "honda"').show()

print('\n', 'Exibindo a média de HP dos carros agrupados por marca, onde marcas avaliadas são "honda" e "bmw"...', '\n')
spSession.sql('select make, cast(avg(hp) as decimal(10,2)) as hp from rowTB1 where make IN ("honda","bmw") group by make').show()

Disparando uma consulta contra a tabela temporária <rowTB1>... 

Exibindo todos os registros que sejam da marca "honda"... 

+---------+---+-----+
|     body| hp| make|
+---------+---+-----+
|hatchback| 60|honda|
|hatchback| 58|honda|
|hatchback| 76|honda|
|hatchback| 76|honda|
|hatchback| 76|honda|
|    sedan| 76|honda|
|    wagon| 76|honda|
|hatchback| 86|honda|
|    sedan| 86|honda|
|hatchback| 86|honda|
|    sedan| 86|honda|
|    sedan|100|honda|
|    sedan|101|honda|
+---------+---+-----+


 Exibindo a média de HP dos carros agrupados por marca, onde marcas avaliadas são "honda" e "bmw"... 

+-----+------+
| make|    hp|
+-----+------+
|  bmw|138.88|
|honda| 80.23|
+-----+------+



### Importando arquivo CSV e criando o DataFrame (diretamente)...

In [39]:
#file ="/home/daniellj/Projetos/Github/DistributedComputing/ApacheSpark/datasets/cars.csv"
file = "C:\\Users\\whitecube.daniel\\Projetos_Daniel\\DistributedComputing\\Datasets\\cars.csv"

rowRDD5 = spSession.read.csv(file, header=True) # header -> informando que o arquivo tem cabeçalho

print('Tipo do objeto criado <carsDataFrame>:', type(rowRDD5), '\n')
print('Imprimindo as 10 primeiras linhas do DataFrame criado...', '\n')
rowRDD5.show(10)

Tipo do objeto criado <carsDataFrame>: <class 'pyspark.sql.dataframe.DataFrame'> 

Imprimindo as 10 primeiras linhas do DataFrame criado... 

+----------+--------+------+-----+---------+-----+---------+---+----+--------+-------+-----+
|      MAKE|FUELTYPE|ASPIRE|DOORS|     BODY|DRIVE|CYLINDERS| HP| RPM|MPG-CITY|MPG-HWY|PRICE|
+----------+--------+------+-----+---------+-----+---------+---+----+--------+-------+-----+
|    subaru|     gas|   std|  two|hatchback|  fwd|     four| 69|4900|      31|     36| 5118|
| chevrolet|     gas|   std|  two|hatchback|  fwd|    three| 48|5100|      47|     53| 5151|
|     mazda|     gas|   std|  two|hatchback|  fwd|     four| 68|5000|      30|     31| 5195|
|    toyota|     gas|   std|  two|hatchback|  fwd|     four| 62|4800|      35|     39| 5348|
|mitsubishi|     gas|   std|  two|hatchback|  fwd|     four| 68|5500|      37|     41| 5389|
|     honda|     gas|   std|  two|hatchback|  fwd|     four| 60|5500|      38|     42| 5399|
|    nissan|     gas|

### Com o DataFrame criado, agora novamente iremos criar a TABELA TEMPORÁRIA para poder usar a manipulação com linguagem SQL ANSI...

In [40]:
rowRDD5.createOrReplaceTempView("rowTB2")

### Com a tabela temporária criada, podemos disparar comandos SQL-ANSI sobre o objeto criado...

In [45]:
rowTB3 = spSession.sql('select make, RPM, price from rowTB2 where make IN ("subaru", "dodge") order by make ASC, price ASC')
rowTB3.show()

+------+----+-----+
|  make| RPM|price|
+------+----+-----+
| dodge|5000|12964|
| dodge|5500| 5572|
| dodge|5500| 6229|
| dodge|5500| 6377|
| dodge|5500| 6692|
| dodge|5500| 7609|
| dodge|5500| 7957|
| dodge|5000| 8921|
|subaru|5200|10198|
|subaru|4800|11259|
|subaru|4800|11694|
|subaru|4900| 5118|
|subaru|4400| 7053|
|subaru|4800| 7126|
|subaru|4800| 7463|
|subaru|4400| 7603|
|subaru|4400| 7775|
|subaru|4800| 8013|
|subaru|4800| 9233|
|subaru|5200| 9960|
+------+----+-----+

